In [ ]:
# -*- coding: utf-8 -*-
# -*- author : Vincent Roduit -*-
# -*- date : 2023-11-25 -*-
# -*- Last revision: 2023-11-25 -*-
# -*- python version : 3.11.6 -*-
# -*- Description: Notebook that summarize results-*-

# <center> CS -433 Machine Learning </center>
## <center> Ecole Polytechnique Fédérale de Lausanne </center>
### <center>Road Segmentation </center>
---

### Preparing environment for Google Colaboratory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/ml-project-2-team-slo/source

### Imports

In [3]:
#import libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

#import model parameters
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
from torchvision import models

%load_ext autoreload
%autoreload 2

In [1]:
#import files
from data_processing import*
from visualization import visualize, visualize_patch
import constants
from test_data import TestData

#import models
from cnn import Basic_CNN
from logistic_regression import LogisticRegression
#from vgg16 import CustomVGG16

In [4]:
# Set random seed for reproducibility
torch.manual_seed(0)

## 1. Data wrangling and visualization

In [13]:
myDatas = AdvancedProcessing(standardize=False)
myDatas.proceed()

Loading data...
Done!
Splitting data...
Done!
Creating patches...
Creating patches for training set...
Create sample 0/100
Creating patches for validation set...
Done!
Creating dataloader...
Done!


## 2. Define and train models

### 2.1 Logistic regression

A first attempt could be to try with some linear model. The first approach here is to use a simple logistic regression. In order to use a logistic regression, one need to extract feature from the image. A choice could be to use the mean and the standard deviation as features. The following section will present these approach.

In [ ]:
LogisticData = BasicProcessing()
LogisticData.load_data()
LogisticData.create_patches()

In [ ]:
LogReg = LogisticRegression(LogisticData.imgs_patches, LogisticData.gt_imgs_patches)
LogReg.compute_vectors()

In [ ]:
plt.scatter(LogReg.X[:, 0], LogReg.X[:, 1], c=LogReg.Y, edgecolors="k", cmap=plt.cm.Paired)
plt.show()

A problem already arises. The datas are not linearly separable. Let's still try to train the model.

In [ ]:
LogReg.train()
LogReg.predict()

In [ ]:
print(f'From this model, the accuracy is {LogReg.accuracy*100:.2f}% and the F1 score is {LogReg.f1*100:.2f}%')

The unsatisfactory results tend us to move to Convolutional Networks, which are more suitable for image datas.

### 2.2 Basic Convolutional Neural Network

In [15]:
cnn = Basic_CNN(constants.WINDOW_SIZE)

# Define the loss function, optimizer and scheduler
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
criterion = nn.BCEWithLogitsLoss()

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    criterion,
    myDatas.train_dataloader,
    myDatas.validate_dataloader,
    num_epochs=10)

Epoch 1/10, Loss: 0.6778785529708863, Validation Accuracy: 0.7355, F1 score: nan
Epoch 2/10, Loss: 0.6702804518699645, Validation Accuracy: 0.7355, F1 score: nan


KeyboardInterrupt: 